# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [13]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Clean the Data SEt 

In [3]:
spaceship.isna().sum()
#your code here
spaceship.dropna(inplace=True)
spaceship.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
# Transform the 'Cabin' column to be less granular
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: str(x)[0] if pd.notna(x) else x)
spaceship.drop(columns=['PassengerId', 'Name'], inplace=True)
# Convert non-numerical columns to dummy variables
spaceship = pd.get_dummies(spaceship, columns=spaceship.select_dtypes(include=['object', 'bool']).columns)
spaceship



,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Transported_False,Transported_True
0,39.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,...,0,0,0,0,0,1,1,0,1,0
1,24.0,109.0,9.0,25.0,549.0,44.0,1,0,0,1,...,1,0,0,0,0,1,1,0,0,1
2,58.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0,1,...,0,0,0,0,0,1,0,1,1,0
3,33.0,0.0,1283.0,371.0,3329.0,193.0,0,1,0,1,...,0,0,0,0,0,1,1,0,1,0
4,16.0,303.0,70.0,151.0,565.0,2.0,1,0,0,1,...,1,0,0,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0,0,1,0,1,...,0,0,0,1,0,0,0,1,1,0
8689,18.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,...,0,1,0,0,1,0,1,0,1,0
8690,26.0,0.0,0.0,1872.0,1.0,0.0,1,0,0,1,...,0,1,0,0,0,1,1,0,0,1
8691,32.0,0.0,1049.0,0.0,353.0,3235.0,0,1,0,1,...,0,0,0,1,0,0,1,0,1,0


**Perform Train Test Split**

In [12]:
# Define the features and the target variable
X = spaceship.drop(columns=['Transported_True', 'Transported_False'])
y = spaceship['Transported_True']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the first few rows of each split
print("Training Features:")
print(X_train.head())
print("\nTesting Features:")
print(X_test.head())
print("\nTraining Target:")
print(y_train.head())
print("\nTesting Target:")
print(y_test.head())

Training Features:
       Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
7832  25.0          0.0     1673.0           0.0   642.0   612.0   
5842  36.0          0.0     2624.0        1657.0  2799.0     1.0   
3928  34.0          0.0        0.0           0.0     0.0     0.0   
4091  37.0          0.0        0.0           0.0     0.0     0.0   
7679  22.0          0.0        0.0           0.0     0.0     0.0   

      HomePlanet_Earth  HomePlanet_Europa  HomePlanet_Mars  CryoSleep_False  \
7832                 0                  1                0                1   
5842                 0                  1                0                1   
3928                 0                  0                1                0   
4091                 1                  0                0                0   
7679                 1                  0                0                0   

      ...  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  \
7832  ...        0        0        0   

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [14]:
# Apply Bagging with a Decision Tree classifier
bagging_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=100,
    bootstrap=True,
    random_state=42
)
bagging_clf.fit(X_train, y_train)
y_pred_bagging = bagging_clf.predict(X_test)
bagging_accuracy = accuracy_score(y_test, y_pred_bagging)

# Apply Pasting with a Decision Tree classifier
pasting_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=100,
    bootstrap=False,
    random_state=42
)
pasting_clf.fit(X_train, y_train)
y_pred_pasting = pasting_clf.predict(X_test)
pasting_accuracy = accuracy_score(y_test, y_pred_pasting)

# Output the accuracy results
print(f"Bagging Accuracy: {bagging_accuracy * 100:.2f}%")
print(f"Pasting Accuracy: {pasting_accuracy * 100:.2f}%")


Bagging Accuracy: 79.35%
Pasting Accuracy: 79.73%


- Random Forests

In [15]:
from sklearn.ensemble import RandomForestClassifier

# Apply Random Forest with a set number of estimators (trees)
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_clf.fit(X_train, y_train)

# Predict the target variable on the test set
y_pred_rf = random_forest_clf.predict(X_test)

# Calculate the accuracy
random_forest_accuracy = accuracy_score(y_test, y_pred_rf)

# Output the accuracy result
print(f"Random Forest Accuracy: {random_forest_accuracy * 100:.2f}%")

Random Forest Accuracy: 81.09%


- Gradient Boosting

In [16]:
from sklearn.ensemble import GradientBoostingClassifier

# Apply Gradient Boosting with a set number of estimators
gradient_boosting_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gradient_boosting_clf.fit(X_train, y_train)

# Predict the target variable on the test set
y_pred_gb = gradient_boosting_clf.predict(X_test)

# Calculate the accuracy
gradient_boosting_accuracy = accuracy_score(y_test, y_pred_gb)

# Output the accuracy result
print(f"Gradient Boosting Accuracy: {gradient_boosting_accuracy * 100:.2f}%")


Gradient Boosting Accuracy: 81.01%


- Adaptive Boosting

In [17]:
from sklearn.ensemble import AdaBoostClassifier

# Apply Adaptive Boosting with a set number of estimators
adaboost_clf = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1), 
    n_estimators=100, 
    learning_rate=0.5, 
    random_state=42
)
adaboost_clf.fit(X_train, y_train)

# Predict the target variable on the test set
y_pred_ab = adaboost_clf.predict(X_test)

# Calculate the accuracy
adaboost_accuracy = accuracy_score(y_test, y_pred_ab)

# Output the accuracy result
print(f"Adaptive Boosting Accuracy: {adaboost_accuracy * 100:.2f}%")


Adaptive Boosting Accuracy: 79.27%


Which model is the best and why?

In [18]:
# Determine the best model
accuracies = {
    'Bagging': bagging_accuracy,
    'Pasting': pasting_accuracy,
    'Random Forest': random_forest_accuracy,
    'Gradient Boosting': gradient_boosting_accuracy,
    'AdaBoost': adaboost_accuracy
}

best_model = max(accuracies, key=accuracies.get)
best_accuracy = accuracies[best_model]

print(f"\nThe best model is {best_model} with an accuracy of {best_accuracy * 100:.2f}%")



The best model is Random Forest with an accuracy of 81.09%
